Data [Section, Change from Previous Month, Previous Month’s Payment, Previous Month’s Attendance, Previous Month’s Dungeon Play, Previous Month’s Raid Play, This Month’s Attendance (target), This Month’s Payment (target)]:

1. Confusion Matrix (Correlation)
2. Regression Analysis (Next Month’s Attendance, Next Month’s Payment) - DT, RF, ET, XGBoost, SVM
3. Classification Analysis (User Group) - Clustering (Visualization confirmed) - DT, RF, ET, XGBoost, SVM

In [2]:
import pandas as pd

In [3]:
# Load data
df2022 = pd.read_excel("./data/231115_2022.xlsx")

In [4]:
# Check data dimensions
df2022.shape

(16800, 98)

In [5]:
# Inspect the top 5 rows
df2022.head()

,유저고유값,캐릭터고유값,cb_a2201,cb_a2202,cb_a2203,cb_a2204,cb_a2205,cb_a2206,cb_a2207,cb_a2208,...,m11_dun1,m11_dun2,m11_dun3,m11_dun4,m11_dunq,m12_dun1,m12_dun2,m12_dun3,m12_dun4,m12_dunq
0,4205,123226681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11970,3148,0.0,0.0,0.0,1488270.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15570,112904343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126057.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15570,116003030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18526,71318869,0.0,36306.0,0.0,0.0,0.0,0.0,0.0,36306.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Check for missing values
df2022.isna().sum()

유저고유값           0
캐릭터고유값          0
cb_a2201       58
cb_a2202       41
cb_a2203       35
            ...  
m12_dun1    14742
m12_dun2    15522
m12_dun3    15720
m12_dun4    16613
m12_dunq    16613
Length: 98, dtype: int64

In [7]:
# Fill missing values
df2022.fillna(0, inplace = True)

In [8]:
# Inspect the top 5 rows
df2022.head()

,유저고유값,캐릭터고유값,cb_a2201,cb_a2202,cb_a2203,cb_a2204,cb_a2205,cb_a2206,cb_a2207,cb_a2208,...,m11_dun1,m11_dun2,m11_dun3,m11_dun4,m11_dunq,m12_dun1,m12_dun2,m12_dun3,m12_dun4,m12_dunq
0,4205,123226681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11970,3148,0.0,0.0,0.0,1488270.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15570,112904343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126057.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15570,116003030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18526,71318869,0.0,36306.0,0.0,0.0,0.0,0.0,0.0,36306.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Process character unique ID
df2022["캐릭터고유값"] = range(len(df2022["캐릭터고유값"]))

In [10]:
# Extract columns from DataFrame
# df2021_power = df2021[["cb_a2101", "cb_a2102", "cb_a2103", "cb_a2104", "cb_a2105", "cb_a2106", "cb_a2107", "cb_a2108", "cb_a2109", "cb_a2110", "cb_a2111", "cb_a2111"]]
df2022_power = df2022.iloc[:, 2:14]

Combat Power: 2 ~ 13 <br>
Attendance: 14 ~ 25<br>
Payment: 26 ~ 37<br>
Dungeon: 42, 47, 52, 57, 62, 67, 72, 77, 82, 87, 92, 97, 98:122

Character - Combat Power Difference, Dungeon, Combat Power Level (Categorical) -> Next Month’s Combat Power Level (Categorical)

In [11]:
def convert_power(x):
    if x <= 100000:
        x = 1
    elif (x > 100000) and (x <= 500000):
        x = 2
    elif (x > 500000) and (x <= 1000000):
        x = 3
    elif (x > 1000000) and (x <= 2500000):
        x = 4
    elif (x > 2500000) and (x <= 5000000):
        x = 5
    elif (x > 5000000) and (x <= 7500000):
        x = 6
    elif (x > 7500000):
        x = 7
    else:
        print("power error")
    
    return x

In [12]:
df2022.columns

Index(['유저고유값', '캐릭터고유값', 'cb_a2201', 'cb_a2202', 'cb_a2203', 'cb_a2204',
       'cb_a2205', 'cb_a2206', 'cb_a2207', 'cb_a2208', 'cb_a2209', 'cb_a2210',
       'cb_a2211', 'cb_a2212', 'a2201', 'a2202', 'a2203', 'a2204', 'a2205',
       'a2206', 'a2207', 'a2208', 'a2209', 'a2210', 'a2211', 'a2212', 'paid01',
       'paid02', 'paid03', 'paid04', 'paid05', 'paid06', 'paid07', 'paid08',
       'paid09', 'paid10', 'paid11', 'paid12', 'm01_dun1', 'm01_dun2',
       'm01_dun3', 'm01_dun4', 'm01_dunq', 'm02_dun1', 'm02_dun2', 'm02_dun3',
       'm02_dun4', 'm02_dunq', 'm03_dun1', 'm03_dun2', 'm03_dun3', 'm03_dun4',
       'm03_dunq', 'm04_dun1', 'm04_dun2', 'm04_dun3', 'm04_dun4', 'm04_dunq',
       'm05_dun1', 'm05_dun2', 'm05_dun3', 'm05_dun4', 'm05_dunq', 'm06_dun1',
       'm06_dun2', 'm06_dun3', 'm06_dun4', 'm06_dunq', 'm07_dun1', 'm07_dun2',
       'm07_dun3', 'm07_dun4', 'm07_dunq', 'm08_dun1', 'm08_dun2', 'm08_dun3',
       'm08_dun4', 'm08_dunq', 'm09_dun1', 'm09_dun2', 'm09_dun3', 'm

In [14]:
df2022["m01_dg_low"] = df2022["m01_dun1"] + df2022["m01_dun2"]
df2022["m01_dg_high"] = df2022["m01_dun3"] + df2022["m01_dun4"]

df2022["m02_dg_low"] = df2022["m02_dun1"] + df2022["m02_dun2"]
df2022["m02_dg_high"] = df2022["m02_dun3"] + df2022["m02_dun4"]

df2022["m03_dg_low"] = df2022["m03_dun1"] + df2022["m03_dun2"]
df2022["m03_dg_high"] = df2022["m03_dun3"] + df2022["m03_dun4"]

df2022["m04_dg_low"] = df2022["m04_dun1"] + df2022["m04_dun2"]
df2022["m04_dg_high"] = df2022["m04_dun3"] + df2022["m04_dun4"]

df2022["m05_dg_low"] = df2022["m05_dun1"] + df2022["m05_dun2"]
df2022["m05_dg_high"] = df2022["m05_dun3"] + df2022["m05_dun4"]

df2022["m06_dg_low"] = df2022["m06_dun1"] + df2022["m06_dun2"]
df2022["m06_dg_high"] = df2022["m06_dun3"] + df2022["m06_dun4"]

df2022["m07_dg_low"] = df2022["m07_dun1"] + df2022["m07_dun2"]
df2022["m07_dg_high"] = df2022["m07_dun3"] + df2022["m07_dun4"]

df2022["m08_dg_low"] = df2022["m08_dun1"] + df2022["m08_dun2"]
df2022["m08_dg_high"] = df2022["m08_dun3"] + df2022["m08_dun4"]

df2022["m09_dg_low"] = df2022["m09_dun1"] + df2022["m09_dun2"]
df2022["m09_dg_high"] = df2022["m09_dun3"] + df2022["m09_dun4"]

df2022["m10_dg_low"] = df2022["m10_dun1"] + df2022["m10_dun2"]
df2022["m10_dg_high"] = df2022["m10_dun3"] + df2022["m10_dun4"]

df2022["m11_dg_low"] = df2022["m11_dun1"] + df2022["m11_dun2"]
df2022["m11_dg_high"] = df2022["m11_dun3"] + df2022["m11_dun4"]

df2022["m12_dg_low"] = df2022["m12_dun1"] + df2022["m12_dun2"]
df2022["m12_dg_high"] = df2022["m12_dun3"] + df2022["m12_dun4"]



In [15]:
# Create a DataFrame for each character
df_character = pd.DataFrame()

for idx_character in range(df2022.shape[0]):
    for idx_power in range(3, 14):
        if (df2022.iloc[idx_character, idx_power] != 0) and (df2022.iloc[idx_character, idx_power - 1] != 0):
            power_diff = df2022.iloc[idx_character, idx_power] - df2022.iloc[idx_character, idx_power - 1]
            
            # 1,2 (Jan, Feb)
            if idx_power == 3:
                dg_low = df2022.iloc[idx_character, 98]
                dg_high = df2022.iloc[idx_character, 99]
                dg_quest = df2022.iloc[idx_character, 42]
                
            # 2,3
            elif idx_power == 4:
                dg_low = df2022.iloc[idx_character, 100]
                dg_high = df2022.iloc[idx_character, 101]
                dg_quest = df2022.iloc[idx_character, 47]
                
            # 3,4
            elif idx_power == 5:
                dg_low = df2022.iloc[idx_character, 102]
                dg_high = df2022.iloc[idx_character, 103]
                dg_quest = df2022.iloc[idx_character, 52]
                
            # 4,5
            elif idx_power == 6:
                dg_low = df2022.iloc[idx_character, 104]
                dg_high = df2022.iloc[idx_character, 105]
                dg_quest = df2022.iloc[idx_character, 57]
                
            # 5,6
            elif idx_power == 7:
                dg_low = df2022.iloc[idx_character, 106]
                dg_high = df2022.iloc[idx_character, 107]
                dg_quest = df2022.iloc[idx_character, 62]
                
            # 6,7
            elif idx_power == 8:
                dg_low = df2022.iloc[idx_character, 108]
                dg_high = df2022.iloc[idx_character, 109]
                dg_quest = df2022.iloc[idx_character, 67]
                
            # 7,8
            elif idx_power == 9:
                dg_low = df2022.iloc[idx_character, 110]
                dg_high = df2022.iloc[idx_character, 111]
                dg_quest = df2022.iloc[idx_character, 72]
                
            # 8,9
            elif idx_power == 10:
                dg_low = df2022.iloc[idx_character, 112]
                dg_high = df2022.iloc[idx_character, 113]
                dg_quest = df2022.iloc[idx_character, 77]
            # 9,10
            elif idx_power == 11:
                dg_low = df2022.iloc[idx_character, 114]
                dg_high = df2022.iloc[idx_character, 115]
                dg_quest = df2022.iloc[idx_character, 82]
                
            # 10,11
            elif idx_power == 12:
                dg_low = df2022.iloc[idx_character, 116]
                dg_high = df2022.iloc[idx_character, 117]
                dg_quest = df2022.iloc[idx_character, 87]
                
            # 11,12
            elif idx_power == 13:
                dg_low = df2022.iloc[idx_character, 118]
                dg_high = df2022.iloc[idx_character, 119]
                dg_quest = df2022.iloc[idx_character, 92]
            
            current_power_level = convert_power(df2022.iloc[idx_character, idx_power - 1])
            next_power_level = convert_power(df2022.iloc[idx_character, idx_power])
                
        else:
            continue
        
        tmp_df = pd.DataFrame({"power_difference": power_diff, "low_dungeon": dg_low, "high_dungeon": dg_high,
                               "quest_dungeon": dg_quest, "current_power_level":current_power_level, "next_power_level":next_power_level}, index = [0])
        df_character = pd.concat([df_character, tmp_df], axis = 0)

In [16]:
# Reset the DataFrame index
df_character = df_character.reset_index(drop=True)

In [17]:
# Inspect the top 5 rows
df_character.head(20)

,power_difference,low_dungeon,high_dungeon,quest_dungeon,current_power_level,next_power_level
0,203676.0,1.0,0.0,0.0,1,2
1,-95113.2,4.0,0.0,0.0,2,2
2,0.0,0.0,0.0,0.0,3,3
3,0.0,0.0,0.0,0.0,3,3
4,0.0,0.0,0.0,0.0,3,3
5,0.0,0.0,0.0,0.0,3,3
6,0.0,4.0,2.0,0.0,3,3
7,0.0,0.0,0.0,0.0,2,2
8,0.0,0.0,0.0,0.0,2,2
9,-56277.6,0.0,0.0,0.0,2,2


In [18]:
# check the distribution of current power levels
df_character["current_power_level"].value_counts().sort_index()

current_power_level
1    1211
2    2115
3    2377
4    3005
5    2179
6    1343
7     536
Name: count, dtype: int64

In [19]:
# check the distribution of next power levels
df_character["next_power_level"].value_counts().sort_index()

next_power_level
1    1048
2    2110
3    2324
4    3061
5    2191
6    1418
7     614
Name: count, dtype: int64

In [20]:
df_character.to_csv("./preprocessed_data/df2022_character.csv", index=False, encoding="utf-8-sig")

Per user - Payment, Attendance, Number of secondary characters, Average combat power level -> Next month’s payment or attendance (numeric)

In [21]:
# Remove duplicate rows based on the user unique ID
df2022_unique = df2022.drop_duplicates(subset='유저고유값', keep='first', ignore_index=True)

In [22]:
# Check data dimensions
df2022_unique.shape

(12750, 122)

In [23]:
# Create a column for the number of secondary characters per user
df2022_unique = pd.merge(df2022_unique, df2022['유저고유값'].value_counts().reset_index(), how='left', on='유저고유값')

In [24]:
# Create a column for the average combat power of secondary characters per user
tmp_df2022 = df2022.iloc[:, :14].drop('캐릭터고유값', axis = 1)

tmp_df2022 = tmp_df2022.groupby("유저고유값").mean().reset_index()
tmp_df2022.columns = ["유저고유값", "avg_power_01", "avg_power_02", "avg_power_03", "avg_power_04", "avg_power_05", "avg_power_06",
                      "avg_power_07", "avg_power_08", "avg_power_09", "avg_power_10", "avg_power_11", "avg_power_12"]

df2022_unique = pd.merge(df2022_unique, tmp_df2022, how = 'left', on = '유저고유값')

In [25]:
# Create a DataFrame for each character
df_user = pd.DataFrame()

for idx_user in range(df2022_unique.shape[0]):
    for idx_power in range(123, 135):
        # 1 (Jan)
        if idx_power == 123:
            attendance = df2022_unique.iloc[idx_user, 14]             # This month's attendance
            paid = df2022_unique.iloc[idx_user, 26]                   # This month's payment
            nCharacter = df2022_unique.iloc[idx_user, 122]            # Number of secondary characters
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])   # Average combat power level of secondary characters
            next_attendance = df2022_unique.iloc[idx_user, 15]        # Next month's attendance
            next_paid = df2022_unique.iloc[idx_user, 27]              # Next month's payment
            
        # 2
        elif idx_power == 124:
            attendance = df2022_unique.iloc[idx_user, 15]
            paid = df2022_unique.iloc[idx_user, 27]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 16]
            next_paid = df2022_unique.iloc[idx_user, 28]
            
        # 3
        elif idx_power == 125:
            attendance = df2022_unique.iloc[idx_user, 16]
            paid = df2022_unique.iloc[idx_user, 28]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 17]
            next_paid = df2022_unique.iloc[idx_user, 29]
            
        # 4
        elif idx_power == 126:
            attendance = df2022_unique.iloc[idx_user, 17]
            paid = df2022_unique.iloc[idx_user, 29]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 18]
            next_paid = df2022_unique.iloc[idx_user, 30]
            
        # 5
        elif idx_power == 127:
            attendance = df2022_unique.iloc[idx_user, 18]
            paid = df2022_unique.iloc[idx_user, 30]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 19]
            next_paid = df2022_unique.iloc[idx_user, 31]
            
        # 6
        elif idx_power == 128:
            attendance = df2022_unique.iloc[idx_user, 19]
            paid = df2022_unique.iloc[idx_user, 31]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 20]
            next_paid = df2022_unique.iloc[idx_user, 32]
            
        # 7
        elif idx_power == 129:
            attendance = df2022_unique.iloc[idx_user, 20]
            paid = df2022_unique.iloc[idx_user, 32]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 21]
            next_paid = df2022_unique.iloc[idx_user, 33]
            
        # 8
        elif idx_power == 130:
            attendance = df2022_unique.iloc[idx_user, 21]
            paid = df2022_unique.iloc[idx_user, 33]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 22]
            next_paid = df2022_unique.iloc[idx_user, 34]
            
        # 9
        elif idx_power == 131:
            attendance = df2022_unique.iloc[idx_user, 22]
            paid = df2022_unique.iloc[idx_user, 34]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 23]
            next_paid = df2022_unique.iloc[idx_user, 35]
            
        # 10
        elif idx_power == 132:
            attendance = df2022_unique.iloc[idx_user, 23]
            paid = df2022_unique.iloc[idx_user, 35]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 24]
            next_paid = df2022_unique.iloc[idx_user, 36]
            
        # 11
        elif idx_power == 133:
            attendance = df2022_unique.iloc[idx_user, 24]
            paid = df2022_unique.iloc[idx_user, 36]
            nCharacter = df2022_unique.iloc[idx_user, 122]
            power_level = convert_power(df2022_unique.iloc[idx_user, idx_power])
            next_attendance = df2022_unique.iloc[idx_user, 25]
            next_paid = df2022_unique.iloc[idx_user, 37]
        
        tmp_df = pd.DataFrame({"current_attendance": attendance, "current_paid": paid, "num_characters": nCharacter,
                               "power_level": power_level, "next_attendance": next_attendance, "next_paid": next_paid}, index = [0])
        df_user = pd.concat([df_user, tmp_df], axis = 0)

In [26]:
# Reset the DataFrame index
df_user = df_user.reset_index(drop=True)

In [27]:
# Inspect the top 5 rows
df_user.head()

,current_attendance,current_paid,num_characters,power_level,next_attendance,next_paid
0,0,0.0,1,1,0,0.0
1,0,0.0,1,1,0,0.0
2,0,0.0,1,1,0,0.0
3,0,0.0,1,1,0,0.0
4,0,0.0,1,1,0,0.0


In [28]:
# check the distribution of next attendances
df_user["next_attendance"].value_counts().sort_index()

next_attendance
0     116700
1      12486
2       4926
3       2822
4       1981
5       1541
6       1190
7        989
8        777
9        662
10       598
11       520
12       488
13       420
14       387
15       389
16       336
17       300
18       286
19       295
20       264
21       261
22       273
23       235
24       272
25       294
26       300
27       339
28       515
29       497
30       820
31       837
Name: count, dtype: int64

In [40]:
# check the distribution of next_paid
df_user["next_paid"].value_counts().sort_index()

next_paid
0.0          142596
150.0            49
200.0            27
300.0            25
400.0             6
              ...  
3799000.0         1
5054600.0         1
6127700.0         1
6974000.0         1
9294100.0         1
Name: count, Length: 2732, dtype: int64

In [30]:
df_user.to_csv("./preprocessed_data/df2022_user.csv", index=False, encoding="utf-8-sig")